<a href="https://colab.research.google.com/github/caddey-ai/caddey-examples/blob/main/notebooks/semantic_kernel_caddey_integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semantic Kernel with Caddey Integration

This notebook demonstrates how to integrate Caddey tools into a Semantic Kernel agent using the Model Context Protocol (MCP) and OAuth 2.0 Device Code flow.

## Overview

Caddey exposes app features as **tools**, and Semantic Kernel agents can dynamically discover and invoke these tools via the **Model Context Protocol (MCP)**. This example uses the **OAuth 2.0 Device Code Flow**, ideal for **CLI-based agents** and environments where browser redirects aren't feasible.

## Prerequisites

- **Caddey OAuth Client ID** (create a **Public** OAuth client in Caddey)
- **OpenAI API Key** for the LLM
- Python packages: `semantic-kernel[mcp]`, `requests`, `python-dotenv`

## Step 1: Install Required Packages

First, let's install all the necessary Python packages.

In [ ]:
!pip install -q semantic-kernel[mcp] requests python-dotenv openai

## Step 2: Set Up Environment Variables

You'll need to provide your Caddey OAuth Client ID and OpenAI API Key. In Colab, you can use the secrets manager or set them directly.

In [ ]:
import os
from getpass import getpass

# Set your Caddey OAuth Client ID
if 'CADDEY_CLIENT_ID' not in os.environ:
    os.environ['CADDEY_CLIENT_ID'] = getpass('Enter your Caddey OAuth Client ID: ')

# Set your OpenAI API Key
if 'OPENAI_API_KEY' not in os.environ:
    os.environ['OPENAI_API_KEY'] = getpass('Enter your OpenAI API Key: ')

print("✅ Environment variables set!")

---

### ⚠️ Important: Public Client Required

The OAuth 2.0 Device Code Flow (RFC 8628) **only works with Public clients**. When creating your OAuth client in Caddey, you must select **Public** (not Confidential).

**Confidential clients will receive an `unauthorized_client` error when attempting to use Device Authorization Grant.**

---

## Step 3: Authenticate with OAuth 2.0 Device Code Flow

This step initiates the OAuth 2.0 Device Code Flow. You'll be prompted to visit a URL and enter a code to authenticate.

In [ ]:
import time
import requests

# OAuth endpoints
DEVICE_URL = "https://auth.caddey.ai/realms/caddey/protocol/openid-connect/auth/device"
TOKEN_URL = "https://auth.caddey.ai/realms/caddey/protocol/openid-connect/token"
CLIENT_ID = os.getenv("CADDEY_CLIENT_ID")

# Request device and user codes
device_response = requests.post(DEVICE_URL, data={"client_id": CLIENT_ID})
device = device_response.json()

print("\n" + "="*60)
print("🔐 AUTHENTICATION REQUIRED")
print("="*60)
print(f"\n📱 To sign in, open this URL in your browser:")
print(f"\n   {device['verification_uri_complete']}")
print(f"\n   Or visit: {device['verification_uri']}")
print(f"   And enter code: {device['user_code']}")
print("\n" + "="*60)
print("⏳ Waiting for authentication...\n")

# Poll for access token
while True:
    token_response = requests.post(
        TOKEN_URL,
        data={
            "grant_type": "urn:ietf:params:oauth:grant-type:device_code",
            "device_code": device["device_code"],
            "client_id": CLIENT_ID,
        },
    )
    token_data = token_response.json()
    
    if "access_token" in token_data:
        CADDEY_TOKEN = token_data["access_token"]
        break
    elif token_data.get("error") == "authorization_pending":
        time.sleep(device.get("interval", 5))
    else:
        raise Exception(f"Token error: {token_data}")

print("✅ Logged in successfully! Token acquired.")

## Step 4: Connect Semantic Kernel to Caddey MCP

Now we'll create a Semantic Kernel instance that connects to Caddey's MCP endpoint and can invoke tools.

In [ ]:
import asyncio
from semantic_kernel import Kernel
from semantic_kernel.connectors.mcp import MCPClient
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.agents import Agent

async def create_caddey_kernel():
    # Create a new Semantic Kernel instance
    kernel = Kernel()

    # Create an MCP client to connect to Caddey's MCP endpoint
    print("🔧 Connecting to Caddey MCP endpoint...")
    mcp_client = MCPClient(
        server_url="https://api.caddey.ai/mcp",
        transport="streamable_http",
        headers={"Authorization": f"Bearer {CADDEY_TOKEN}"},
    )

    # Connect and register Caddey MCP tools as kernel functions
    await mcp_client.connect()
    print("✅ Connected to Caddey MCP!")
    
    print("🔧 Fetching available tools from Caddey...")
    caddey_tools = await mcp_client.get_tools()
    print(f"✅ Found {len(caddey_tools)} tools available!\n")
    
    for tool in caddey_tools:
        kernel.add_function(tool.as_kernel_function())

    # Add an OpenAI LLM
    llm = OpenAIChatCompletion(
        model_id="gpt-4o-mini",
        api_key=os.getenv("OPENAI_API_KEY")
    )
    kernel.add_service(llm)

    return kernel, mcp_client, caddey_tools

# Create the kernel
kernel, mcp_client, available_tools = await create_caddey_kernel()
print("🤖 Semantic Kernel created and ready!")

## Step 5: List Available Tools

Let's see what tools are available in your Caddey account.

In [ ]:
print("📋 Available Caddey Tools:\n")
print("="*60)
for i, tool in enumerate(available_tools, 1):
    print(f"{i}. {tool.name}")
    if hasattr(tool, 'description') and tool.description:
        print(f"   Description: {tool.description}")
    print()

## Step 6: Create a Semantic Kernel Agent

Now let's create an agent that can use the Caddey tools.

In [ ]:
# Create a simple Semantic Kernel agent
agent = Agent(
    kernel=kernel,
    name="CaddeyAgent",
    instructions="You are a helpful assistant with access to Caddey tools. Use the available tools to help users accomplish their tasks."
)

print("🤖 Agent created successfully!")

## Step 7: Use Your Agent

Now you can interact with your Semantic Kernel agent using natural language prompts. The agent will automatically discover and invoke the appropriate Caddey tools based on your requests.

In [ ]:
result = await agent.complete_chat("List all available tools in my Caddey account.")
print("\n" + "="*60)
print("RESULT:")
print("="*60)
print(result)

## Step 8: Cleanup

Don't forget to disconnect from the MCP client when you're done.

In [ ]:
await mcp_client.disconnect()
print("✅ Disconnected from Caddey MCP.")

## Important Notes

- The **Device Code Flow** (RFC 8628) is designed for **public clients** and works seamlessly in CLI or server environments without requiring a web server for OAuth callbacks.
- Public clients do not require a client secret, making this flow ideal for devices that cannot securely store credentials.
- For web-based agents that can handle redirects, use the **Authorization Code Flow** instead.
- Access tokens typically expire after 1 hour. You may need to re-run the authentication step when tokens expire.
- Store your `CADDEY_CLIENT_ID` securely and never commit it to version control.

## Additional Resources

- [Semantic Kernel MCP Documentation](https://learn.microsoft.com/en-us/semantic-kernel/concepts/plugins/adding-mcp-plugins)
- [Authlib Documentation](https://docs.authlib.org)
- [OAuth 2.0 Device Authorization Grant (RFC 8628)](https://datatracker.ietf.org/doc/html/rfc8628)
- [MCP Specification](https://modelcontextprotocol.io/specification)
- [Caddey Documentation](https://caddey.ai/docs)

## Need Help?

Visit the [Caddey Support Section](https://caddey.ai/support) for assistance.